In [5]:
import ast
import astor
class AstBug:
    def __init__(self, name):
        self.name = name
        
    def apply(self, ast):
        raise NotImplementedError()

class ComparisonOperatorTransformer(ast.NodeTransformer):
    def __init__(self):
        self.mapping = {
            ast.Lt: ast.Gt,
            ast.Gt: ast.Lt,
            ast.LtE: ast.GtE,
            ast.GtE: ast.LtE,
            ast.Eq: ast.NotEq,
            ast.NotEq: ast.Eq,
        }

    def visit_Compare(self, node):
        for comparision in node.ops:
            if type(comparision) in self.mapping:
                node.ops[node.ops.index(comparision)] = self.mapping[type(comparision)]()
        return node

class IncorrectComparisonOperatorBug(AstBug):
    def __init__(self):
        super().__init__("incorrect_comparison_operator")
    
    def apply(self, ast):
        transformer = ComparisonOperatorTransformer()
        transformer.visit(ast)
        return ast

In [7]:
#First step is to parse the script to an AST using the ast module
script = """
x = 5
y = 10
if x > y:
    print("x is greater than y")
else:
    print("x is less than or equal to y")
"""
ast_tree = ast.parse(script)

# create an instance of the IncorrectComparisonOperatorBug
bug = IncorrectComparisonOperatorBug()

# apply the bug to the ast
bugged_ast = bug.apply(ast_tree)

# use the modified ast to generate the modified script
modified_script = astor.to_source(bugged_ast)
print(modified_script)


x = 5
y = 10
if x < y:
    print('x is greater than y')
else:
    print('x is less than or equal to y')

